ベクトルを足さず、コサイン類似度のみで言葉を生成する。

In [1]:
from gensim.models import KeyedVectors
import json

# Word2Vecモデルのロード
model = KeyedVectors.load_word2vec_format('model_neo_nouns_only.bin', binary=True)

with open('word_pairs.json', 'r', encoding='utf-8') as f:
    word_pairs = json.load(f)

In [3]:

word = "天然水"
# model.similar_by_word(word)
lst = model.similar_by_key(word, topn=10)
lst

[('サントリー天然水', 0.7073501944541931),
 ('南アルプスの天然水', 0.6818466186523438),
 ('ナチュラルミネラルウォーター', 0.6715477705001831),
 ('ミネラルウォーター', 0.6250886917114258),
 ('アサヒおいしい水', 0.6134332418441772),
 ('おいしい水', 0.6005318760871887),
 ('缶チューハイ', 0.5496819019317627),
 ('名水', 0.5460743308067322),
 ('ザ・プレミアム・モルツ', 0.5439411997795105),
 ('カルピスウォーター', 0.5420613884925842)]

ベクトルを足したあとに生成した場合

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

vector_diff = [] # ベクトルの差のリスト
for word1, word2 in word_pairs:
    if word1 in model.key_to_index and word2 in model.key_to_index:
        vector_diff.append(model[word1] - model[word2])
        
v = np.concatenate(vector_diff)
kde = gaussian_kde(v)

pre_vec_diff = kde.resample(300).flatten()
vec_generated = model[word] + pre_vec_diff

model.similar_by_vector(vec_generated, topn=10)

[('天然水', 0.7850627303123474),
 ('南アルプスの天然水', 0.5530255436897278),
 ('サントリー天然水', 0.5445462465286255),
 ('ナチュラルミネラルウォーター', 0.4818604588508606),
 ('ミネラルウォーター', 0.46700742840766907),
 ('アサヒおいしい水', 0.45366722345352173),
 ('日本酒', 0.4358205199241638),
 ('おいしい水', 0.43400683999061584),
 ('千枚田', 0.42660924792289734),
 ('清酒', 0.41334670782089233)]

In [15]:
import MeCab
import ipadic

def process_text_japanese(text):
    mecab = MeCab.Tagger("-Owakati")
    parsed_text = mecab.parse(text)
    words = parsed_text.strip().split()
    return words

# 使用例
text = "日本語のテキストを解析します。"
words = process_text_japanese(text)
print(words)


['日本語', 'の', 'テキスト', 'を', '解析', 'し', 'ます', '。']


In [25]:
from collections import Counter


mecab = MeCab.Tagger(ipadic.MECAB_ARGS)
text = "日本語のテキストを解析します。"
parsed_text = mecab.parse(text)
print(parsed_text)

nouns = []
for line in parsed_text.splitlines():
    if "名詞" in line:
        noun = line.split()[0]  # 名詞の単語部分を取得
        nouns.append(noun)
print(nouns)

word_counts = Counter()
word_counts.update(nouns)
print(word_counts)

日本語	名詞,一般,*,*,*,*,日本語,ニホンゴ,ニホンゴ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
テキスト	名詞,一般,*,*,*,*,テキスト,テキスト,テキスト
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
解析	名詞,サ変接続,*,*,*,*,解析,カイセキ,カイセキ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
。	記号,句点,*,*,*,*,。,。,。
EOS

['日本語', 'テキスト', '解析']
Counter({'日本語': 1, 'テキスト': 1, '解析': 1})


In [17]:
import pandas as pd
import pickle
from tqdm import tqdm
from gensim.corpora import WikiCorpus


# ステップ1: データの解析
# wiki_dump_path = 'jawiki-latest-pages-articles.xml.bz2'
wiki_dump_path = 'jawiki-latest-pages-articles.xml.bz2'
wiki_corpus = WikiCorpus(wiki_dump_path, dictionary={})

# ステップ2: 単語の前処理
def process_text(text):
    mecab = MeCab.Tagger("-Owakati")
    parsed_text = mecab.parse(text)
    words = parsed_text.strip().split()
    return words

# ステップ3: 単語の頻度計算
word_counts = Counter()
for text in tqdm(wiki_corpus.get_texts()):
    for w in text:
        word_counts.update(process_text(w))

# ステップ4: 頻度の高い単語のランキング作成
most_common_words = word_counts.most_common()

# ランキングデータを表形式に変換
data = {"Word": [word for word, freq in most_common_words],
        "Frequency": [freq for word, freq in most_common_words]}
df = pd.DataFrame(data)

# 表形式のデータをPickle形式で保存
output_pickle_file = "word_ranking.pkl"  # 保存先Pickleファイル名

with open(output_pickle_file, "wb") as picklefile:
    pickle.dump(df, picklefile)

print(f"Word ranking saved to {output_pickle_file}")


95it [00:44,  2.13it/s]


KeyboardInterrupt: 

In [15]:
import pandas as pd
import pickle
from tqdm import tqdm
from gensim.corpora import WikiCorpus
import MeCab
import ipadic
from collections import Counter


# ステップ1: データの解析
wiki_dump_path = 'jawiki-latest-pages-articles.xml.bz2'
wiki_corpus = WikiCorpus(wiki_dump_path, dictionary={})

# ステップ3: 単語の頻度計算
word_counts = Counter()
count = 0
for text in tqdm(wiki_corpus.get_texts()):
    res = "".join(text)
    mecab = MeCab.Tagger(ipadic.MECAB_ARGS)
    parsed_text = mecab.parse(res)
    # print(parse_text)
    
    nouns = []
    for line in parsed_text.splitlines():
        if "名詞" in line:
            noun = line.split()[0]  # 名詞の単語部分を取得
            nouns.append(noun)
    word_counts.update(nouns)

    count += 1
    if count == 3000:
        break
    # print(word_counts)
m_w = word_counts.most_common()
print(m_w)
    # for ws in text:
        # word_counts.update(process_text(w))

TypeError: WikiCorpus.get_texts() got an unexpected keyword argument 'limit'

In [13]:
# for text in tqdm(wiki_corpus.get_texts()):
#     pass
corpus_length = len(wiki_corpus)
print(f"WikiCorpusの長さ（記事の数）: {corpus_length}")


AttributeError: 'WikiCorpus' object has no attribute 'length'